In [3]:
""""paquetes necesarios para la ejecución del programa"""

import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from scipy.optimize import minimize


""" función para crear el Bell estado aplicando LU parametrica a cada uno de los qubits
el estado es de la forma  U(t1,p1,l1) X U(t0,p0,l1) . (|00> + |11>)/sqrt(2)"""

def Bell_state(t0, p0, l0, t1, p1, l1):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    # qc.x(0)
    qc.u(t0, p0, l0, 0)
    qc.u(t1, p1, l1, 1)
    return Statevector.from_instruction(qc)


""" función para crear el separable_state aplicando LU parametrica a cada uno de los qubits
el estado es de la forma  U(x1,x1,x1) X U(x0,y0,z1) . (|00>)"""

def separable_state(x0, y0, z0, x1, y1, z1):
    
    qc = QuantumCircuit(2)
    qc.u(x0, y0, z0, 0)
    qc.u(x1, y1, z1, 1)
    return Statevector.from_instruction(qc)


""" fubini study  distance"""
def d_fs(x, y):
    return np.arccos(np.real(x.inner(y)))


"""Función de costo: Muestra los parametro para cuando la distacia de flubini study es minima"""

def cost(params):
    t0, p0, l0, t1, p1, l1, x0, y0, z0, x1, y1, z1 = params
    entangled_state = Bell_state(t0, p0, l0, t1, p1, l1)
    separable       = separable_state(x0, y0, z0, x1, y1, z1)
    return d_fs(entangled_state, separable)

# Rango de parámetros
bounds = [
    (0, np.pi),      # t0
    (0, 2*np.pi),    # p0
    (0, 2*np.pi),    # l0
    (0, np.pi),      # t1
    (0, 2*np.pi),    # p1
    (0, 2*np.pi),    # l1
    (0, np.pi),      # x0
    (0, 2*np.pi),    # y0
    (0, 2*np.pi),    # z0
    (0, np.pi),      # x1
    (0, 2*np.pi),    # y1
    (0, 2*np.pi),    # z1
]

# Valores iniciales aleatorios que se encuentran dentro de esos rangos 
x0 = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds])

# Minimización
res = minimize(cost, x0, bounds=bounds, method='L-BFGS-B')

print("Distancia mínima encontrada (radianes):", res.fun)
print("Parámetros óptimos:", res.x)

# Si quieres convertir la distancia mínima a grados:
print("Distancia mínima (grados):", np.degrees(res.fun))


Distancia mínima encontrada (radianes): 0.7853981634053661
Parámetros óptimos: [2.57230172 4.74361735 3.48596858 1.21438624 1.90495987 0.2859682
 0.83024401 3.92898934 1.75434728 3.07666908 2.42138673 0.01012261]
Distancia mínima (grados): 45.00000000045365


In [5]:
# import numpy as np
# from qiskit import QuantumCircuit
# from qiskit.quantum_info import Statevector
# from scipy.optimize import differential_evolution


# # Otro método para minimizar 
# result = differential_evolution(cost, bounds, strategy='best1bin', maxiter=300, popsize=20, tol=1e-6, disp=True)

# # Mostrar resultados
# print("\n📌 Distancia mínima encontrada (radianes):", result.fun)
# print("📌 Distancia mínima (grados):", np.degrees(result.fun))
# print("📌 Parámetros óptimos:")
# for name, value in zip(['t0', 'p0', 'l0', 't1', 'p1', 'l1', 'x0', 'y0', 'z0', 'x1', 'y1', 'z1'], result.x):
#     print(f"{name} = {value:.4f}")


In [6]:
"""Paramétros obtenidos del minimaze"""

(t0, p0, l0, t1, p1, l1)= res.x[6:12]
(x0, y0, z0, x1, y1, z1)= res.x[0:6]
params_00  = res.x[6:12]
theta = res.x

"""Función para encontarar el camino de estados que se encuentran en la geodesica (función de los a puntes del Jefe del grupo)"""

def gamma(k):
    theta =  res.fun
    vector = np.cos(k)*separable_state(x0, y0, z0, x1, y1, z1) + np.sin(k)*(Bell_state(t0, p0, l0, t1, p1, l1) - np.cos(theta)*separable_state(x0, y0, z0, x1, y1, z1) )/(np.sin(theta))
    
    return vector.data

In [ ]:
from qiskit.quantum_info import Operator,Statevector
#############################################################
#  Operators
""" Encontrar el estado más cercano a la geodésica con compuertas nativas.  """


""" Definición de las compuertas nativas donde:
II : la compuerta identidad en cada uno de los qubits
XI : compuerta x en el primer qubit e identidad en el qubit cero
SXI: compuerta sx en el primero, identidad en el qubir cero 
|qubit uno, qubit cero>  =  |0 0 >
"""
II = Operator.from_label("II")
XI = Operator.from_label("XI")
ZI = Operator.from_label("ZI")
SXI= Operator.from_label("XI").power(0.5)

IX = Operator.from_label("IX")
IZ = Operator.from_label("IZ")
ISX= Operator.from_label("IX").power(0.5)

XX = Operator.from_label("XX")
ZZ = Operator.from_label("ZZ")
ISX_ISX = (Operator.from_label("X").power(0.5)).tensor(Operator.from_label("X").power(0.5)) 

###################################################################

"""Vectro inicial |00> """

initial = Statevector.from_label("00")

"""Vector flat (sx en el control y ecr )"""
cir = QuantumCircuit(2)
cir.sx(0)
cir.ecr(0,1)

final = Statevector(cir)


###################################################################

"""Lista de las compuertas nativas sin la ecr"""
""" la lista uno contiene todos los vectores de la palicación de la lista de lacompuerta nativa """
gates_nativas = [II,XI,ZI,SXI,IX,IZ,ISX, XX, ZZ, ISX_ISX]
gates_str =     ["II","XI","ZI","SXI","IX","IZ","ISX", "XX", "ZZ", "ISX_ISX"]

uno = []
for i in gates_nativas:
    U = np.matmul(i.data, initial.data)
    uno.append(U)

"""Distacia en radianes para cada una de las aplicaciones de las compuertas nativas con respecto al estado flat. """
s = 0
for i in uno:
    s += 1
    dis = d_fs(Statevector(i),final)
    print( dis, " ", gates_str[s-1]  )    
#############################################################################  
        

In [ ]:

"""La lsita gate_dos contiene los vectores de estasos pero aplicando dos LU a cada uno de los qubits |00>"""

dos = []
for i in gates_nativas:
    for j in uno:
        UU = np.matmul(i.data,j)   
        dos.append(UU)
            
            
gate_dos = []
for i in gates_str:
    for j in gates_str:
        gate_dos.append([i,j])      
        
        
"""me muestra todos los valores de distancia en radianes y las combinaciones de las compuertas nativas en el estado de inicio |00>"""        
l = 0
for i in dos:
    l += 1
    dis = d_fs(Statevector(i),final)
    print(  dis, "   ",  gate_dos[l-1])                 
            

In [8]:
# para el ghz

# theta(000,GHZ) = 0.7853
# theta(000, W) = 1.57079
# theta(GHZ,W) = 1.57079

"""GHZ state"""

GHZ = QuantumCircuit(3)
GHZ.h(0)
GHZ.cx(0,1)
GHZ.cx(1,2)

ghz = Statevector(GHZ)

"""initila state"""
initial_state = Statevector.from_label("000")

""" state W """
w = Statevector([0,1,1,0,1,0,0,0]/np.linalg.norm(np.array([0,1,1,0,1,0,0,0])))

dis_ghz = d_fs(ghz,initial_state)
dis_w   = d_fs(ghz,w)


print(f"la distancia en grados para el ghz y |000> es: {np.degrees(dis_ghz)},  para el estado w y |000> : {np.degrees(dis_w)}")

print(f"distancia entre GHZ y W :  {np.degrees(d_fs(w,ghz))}")

la distancia en grados para el ghz y |000> es: 45.00000000000001,  para el estado w y |000> : 90.0
distancia entre GHZ y W :  90.0
